In [1]:
import pandas as pd

## Задание 1

In [38]:
log = pd.read_csv('visit_log.csv', sep=';')

In [39]:
def SourceClassifier(file_csv):
    if 'yandex' in file_csv['traffic_source'] or 'google' in file_csv['traffic_source']:
        return 'organic'
    elif ('paid' in file_csv['traffic_source'] or 'email' in file_csv['traffic_source']) and 'Russia' in file_csv['region']:
        return 'ad'
    elif ('paid' in file_csv['traffic_source'] or 'email' in file_csv['traffic_source']) and 'Russia' not in file_csv['region']:
        return 'other'
    else:
        return file_csv['traffic_source']

In [61]:
log['source_type'] = log.apply(SourceClassifier, axis=1)

log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


## Задание 2

In [58]:
url_ = pd.read_csv('URLs.txt')

In [59]:
filt_url = url_[url_['url'].str.contains(r'/\d{8}-', regex=True, case=False)] 

In [60]:
filt_url.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


## Задание 3

In [83]:
rat_dt = pd.read_csv('ratings.csv')

grouped_dt = rat_dt.groupby('userId').count().reset_index()

In [84]:
grouped_dt.head()

,userId,movieId,rating,timestamp
0,1,20,20,20
1,2,76,76,76
2,3,51,51,51
3,4,204,204,204
4,5,100,100,100


In [88]:
ratings_filtered = grouped_dt[grouped_dt['rating'] >= 100]
top_users = ratings_filtered['userId'].tolist()

In [96]:
rat_dt = rat_dt.loc[rat_dt['userId'].isin(top_users)]
rat_dt.head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [92]:
users_lifetime = rat_dt.groupby('userId').agg(['min', 'max'])['timestamp'].reset_index()
users_lifetime.head()

,userId,min,max
0,4,949778714,949982274
1,5,1163373044,1163375145
2,8,1154389340,1154474527
3,15,997937239,1469330735
4,17,1127468587,1127476640


In [97]:
users_lifetime['min'] = pd.to_datetime(users_lifetime['min'], unit = 's')
users_lifetime['max'] = pd.to_datetime(users_lifetime['max'], unit = 's')

def LifeTime(row):
    x = row['max'] - row['min']     
    return x

users_lifetime['lifetime'] = users_lifetime.apply(LifeTime, axis=1)
users_lifetime.head()

,userId,min,max,lifetime
0,4,2000-02-05 19:25:14,2000-02-08 03:57:54,2 days 08:32:40
1,5,2006-11-12 23:10:44,2006-11-12 23:45:45,0 days 00:35:01
2,8,2006-07-31 23:42:20,2006-08-01 23:22:07,0 days 23:39:47
3,15,2001-08-16 04:47:19,2016-07-24 03:25:35,5455 days 22:38:16
4,17,2005-09-23 09:43:07,2005-09-23 11:57:20,0 days 02:14:13


## Задание 4

In [134]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [135]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [137]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [150]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [154]:
table = rzd.merge(auto, how='outer', on='client_id')
table = table.merge(air, how='outer', on='client_id')
full_table = table.merge(client_base['client_id'], how='outer', on='client_id').fillna(0)

full_table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [155]:
#другой вариант
list_df = [client_base, rzd, auto, air]

all_info = pd.concat([d.set_index('client_id') for d in list_df], axis=1).fillna(0).reset_index()

all_info 

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0
